In [1]:
import torch
import torch.nn as nn
from torchvision.datasets import EMNIST
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

In [2]:
number_workers = 0 # number of subprocesses to use for data loading
batch_size = 32
validation_size = 0.2 # percentage of training set to use as validation

train_dataset = EMNIST('./data', 
                       split='balanced',
                       train=True, 
                       download=True, 
                       transform=transforms.ToTensor())

test_dataset = EMNIST('./data', 
                      split='balanced',
                      train=False, 
                      transform=transforms.ToTensor())

train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)

  0%|          | 0/561753746 [00:00<?, ?it/s]

Extracting ./data/EMNIST/raw/gzip.zip to ./data/EMNIST/raw


In [3]:
# obtain the indices that will be used for validation
number_of_train = len(train_dataset)
train_indices = list(range(number_of_train))
np.random.shuffle(train_indices)
splitting = int(np.floor(validation_size * number_of_train))
training_index, validation_index = train_indices[splitting:], train_indices[:splitting]

#  define samplers for obtaining training and validation batches
training_sampler = SubsetRandomSampler(training_index)
validation_sampler = SubsetRandomSampler(validation_index)

# preparation of data loaders
train_loader_batch = DataLoader(train_dataset, batch_size = batch_size, sampler = training_sampler, num_workers = number_workers)
validation_loader_batch = DataLoader(train_dataset, batch_size = batch_size,sampler = validation_sampler, num_workers = number_workers)
test_loader_batch = DataLoader(test_dataset, batch_size = batch_size, num_workers = number_workers)

In [4]:
# define the NN architecture
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()

        # number of hidden neurons in each layer
        hidden_1 = 512
        hidden_2 = 512
        # hidden_3 = 256

        self.fc1 = nn.Linear(28 * 28, hidden_1) # linear layer (28*28=784 -> hidden_1)
        self.bn1 = nn.BatchNorm1d(hidden_1) # batch normalization layer

        self.fc2 = nn.Linear(hidden_1, hidden_2) 
        self.bn2 = nn.BatchNorm1d(hidden_2)

        # self.fc3 = nn.Linear(hidden_2, hidden_3) 
        # self.bn3 = nn.BatchNorm1d(hidden_3) 

        self.fc3 = nn.Linear(hidden_2, 47) # output layer

        self.dropout = nn.Dropout(0.2) # dropout layer

    def forward(self, x):
        # flatten image input
        x = x.view(-1, 28 * 28)

        # add hidden layer, with relu activation function
        x = F.relu(self.bn1(self.fc1(x)))
        # add dropout layer
        x = self.dropout(x)

        # add hidden layer, with relu activation function
        x = F.relu(self.bn2(self.fc2(x)))
        # add dropout layer
        x = self.dropout(x)

        # # add hidden layer, with relu activation function
        # x = F.relu(self.bn3(self.fc3(x)))
        # # add dropout layer
        # x = self.dropout(x)

        # add output layer
        x = self.fc3(x)
        return x

# initialize the NN
model = MLP()
print(model)

MLP(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=512, out_features=47, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [5]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [6]:
# number of epochs to train the model
n_epochs = 10

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train() # prep model for training
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval() # prep model for evaluation
    for data, target in validation_loader_batch:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(validation_loader_batch.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss


print('Training process has finished.')

Epoch: 1 	Training Loss: 1.325321 	Validation Loss: 0.147325
Validation loss decreased (inf --> 0.147325).  Saving model ...
Epoch: 2 	Training Loss: 0.814916 	Validation Loss: 0.112608
Validation loss decreased (0.147325 --> 0.112608).  Saving model ...
Epoch: 3 	Training Loss: 0.698738 	Validation Loss: 0.099987
Validation loss decreased (0.112608 --> 0.099987).  Saving model ...
Epoch: 4 	Training Loss: 0.638826 	Validation Loss: 0.088761
Validation loss decreased (0.099987 --> 0.088761).  Saving model ...
Epoch: 5 	Training Loss: 0.596751 	Validation Loss: 0.084991
Validation loss decreased (0.088761 --> 0.084991).  Saving model ...
Epoch: 6 	Training Loss: 0.566546 	Validation Loss: 0.078980
Validation loss decreased (0.084991 --> 0.078980).  Saving model ...
Epoch: 7 	Training Loss: 0.541298 	Validation Loss: 0.075013
Validation loss decreased (0.078980 --> 0.075013).  Saving model ...
Epoch: 8 	Training Loss: 0.521759 	Validation Loss: 0.071880
Validation loss decreased (0.07501

In [ ]:
testing_loss = 0.0
class_corrected = list(0. for i in range(47))
class_total = list(0. for i in range(47))

model.eval() # preparing the model for evaluation

for data, target in test_loader_batch:
  # forward passing as in this computing the predicted outputs by passing the inputs to the model
    output = model(data) 
    loss = criterion(output,target) # calculating the loss
    testing_loss += loss.item()*data.size(0) # updating the running validation loss
    _, predictions = torch.max(output, 1) # converting the output probabilities to predicted class
    # comparing the predictions to true label
    corrected = np.squeeze(predictions.eq(target.data.view_as(predictions))) 

    for vals in range(len(target)): # calculating the test accuracy for each object class
        label = target.data[vals]
        class_corrected[label] += corrected[vals].item()
        class_total[label] += 1

print('Test Accuracy: %2d%% (%2d out of %2d)' % (100. * np.sum(class_corrected) / np.sum(class_total), np.sum(class_corrected), np.sum(class_total)))

Test Accuracy: 85% (16070 out of 18800)
